# Homework 2: Naive Bayes
## Due September 24th by 11:55 PM

## Legal reasoning (From Murphy, 2.2). (25 pts)

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.  The  prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy. What is wrong with this argument?

### B. (Grad students only)
Making some assumptions of your choice regarding how to model the problem at hand, come up with a more reasonable estimate of the probability of the defendant's guilt.  (There is no right answer here.  You simply need to demonstrate the ability to reason with Bayes' theorem).

In [1]:
#This argument is incorrect because applying bayes theorem we get P(didcrime|bloodtype) = P(bloodtype|didcrime) * P(didcrime)/
                                                                                          # P(probability of bloodtype) 
# this gives us P(didcrime|bloodtype) = 1 * crimerate in area*population / 1/100, so 100/crimerate % * population. So this does
#in fact raise the chance that the suspect did the crime by a factor of 100. For arbitrary numbers, lets say 100,000 people
#live in a town. So plugging it into my formula and using national average murder rates, we get
#P(didcrime|bloodtype) = (1 * 5/10000)/ 1/100. This simplifies to 500/10000 or 5/100 chance he comitted the crime.

## Ham vs Spam (75 pts)
One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, implying that
$$P(X=\mathrm{draw}|Y=\mathrm{spam}) = \frac{27}{25748} = \frac{m_{draw,spam}}{m_{spam}},$$
while in the case of ham, it shows up 5 times so
$$P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{67148} = \frac{m_{draw,ham}}{m_{ham}}.$$
Thus we see that the word 'draw' shows up with a much higher frequency in spam e-mails than in ham.

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [2]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

counts = np.array([v for v in word_dictionary.values()]).sum(axis=0)

Below, I have provided code skeletons. Your job is to make the code skeletons into an operational naive Bayes spam detector. (you may discard these skeletons if you would prefer to code this from scratch). Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:

In [3]:
#What is the prior P(Y=ham) ?
ham_prior = counts[1]/(counts[1] + counts[0])
spam_prior = counts[0] / (counts[1]+counts[0])
print(ham_prior, spam_prior)
# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
spam_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] =  (val[1] / counts[1]) 
    spam_likelihood[key] = (val[0] / counts[0])
    
 

0.7229246247377199 0.2770753752622801


Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [4]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0
    posterior_spam = 1.0
    
    # Loop over all the words in each message
    for w in m:
        # Make sure there isn't any invalid entries or word(s) that haven't been seen before
        try:
            posterior_ham *=  ham_likelihood[w] 
            posterior_spam *=  spam_likelihood[w] 
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)


<ipython-input-4-28e7496bc916>:20: RuntimeWarning: invalid value encountered in double_scalars
  posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
<ipython-input-4-28e7496bc916>:21: RuntimeWarning: invalid value encountered in double_scalars
  posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)


Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [5]:
count_incorrect = 0
count_correct = 0
total = 0
for i,m in enumerate(test_messages):
    spamorham = 0
    spamorham_actual = 0    
    fstper = round(posteriors[i, 0], 4) 
    scdper = round(posteriors[i, 1], 4) 
    
    #check which probabililty is higher
    if scdper > fstper:
        spamorham=1
    
    #was used in testing to give visual of the percentages and what it actual was
    print("Actual: ", test_messages[i][0])
    print("Percent Ham: ", scdper)
    print("Percent Spam: ", fstper, "\n")
    
    #check if it was actually ham and set that as a 1 to test against
    if test_messages[i][0] == 'ham':
        spamorham_actual = 1
    #count how many times we got it correct
    if spamorham == spamorham_actual:
        count_correct += 1
    else:
        count_incorrect += 1
    #increment total to figure out percentage of time we are correct
    total+= 1
    
percentcorrect = (round(count_correct/total, 4)) * 100.0
percentincorrect = (round(count_incorrect/total, 4)) * 100.0
print("Percent correct:", percentcorrect,"\nPercent incorect:", percentincorrect)
    

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9985
Percent Spam:  0.0015 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  spam
Percent Ham:  0.3738
Percent Spam:  0.6262 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.8939
Percent Spam:  0.1061 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9535
Percent Spam:  0.0465 

Actual:  spam
Percent Ham:  0.0001
Percent Spam:  0.9999 

Actual:  ham
Percent Ham:  0.9866
Percent Spam:  0.0134 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent 

Actual:  ham
Percent Ham:  0.5403
Percent Spam:  0.4597 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.999
Percent Spam:  0.001 

Actual:  ham
Percent Ham:  0.9975
Percent Spam:  0.0025 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9704
Percent Spam:  0.0296 

Actual:  ham
Percent Ham:  0.974
Percent Spam:  0.026 

Actual:  ham
Percent Ham:  0.9757
Percent Spam:  0.0243 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  0.9999
Percent Spam:  0.0001 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9999
Percent Spam:  0.0001 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9879
Percent Spam:  0.0121 

Actual:  spam


Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9998
Percent Spam:  0.0002 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9992
Percent Spam:  0.0008 

Actual:  ham
Percent Ham:  0.993
Percent Spam:  0.007 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9715
Percent Spam:  0.0285 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9957
Percent Spam:  0.0043 

Actual:  ham
Percent Ham:  0.9994
Percent Spam:  0.0006 

Actual:  spam
Percent Ham:  0.411
Percent Spam:  0.589 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9974
Percent Spam:  0.0026 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Per

Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9999
Percent Spam:  0.0001 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.9983
Percent Spam:  0.0017 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  0.8497
Percent Spam:  0.1503 

Actual:  spam
Percent Ham:  0.0
Percent Spam:  1.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actual:  ham
Percent Ham:  1.0
Percent Spam:  0.0 

Actu